# Approximate Nearest Neighbor (ANN) Search in PECOS 

PECOS provides the efficient approach for **approximate nearest neighbor (ANN) search**. More specifically, after training an hierarchical navigable small world (HNSW) model (or buildling the **PECOS-HNSW indexer**) with a corpus of vectors, PECOS supports to efficiently infer top-K approximated nearest indexed vectors for an arbitrary query vector. In this part of the tutorial, we will demonstrate how to use PECOS-HNSW tackle the approximate nearest neighbor (ANN) search problem and how to integrate HNSW with PECOS XMR models.

#### HNSW at a glimpse
The search procedure of HNSW can be summarized as:
* traverse from top layer (course-grain graph, long-range link) to bottom layer (fine-grain graph, short-range link)
* best first search traversal on each graph, where the best candidate serves as initial to next layer
<div> <br/><img src=\"imgs/hnsw-example.png\" width=\"80%\"/> </div>


## Highlight of PECOS-HNSW

* Support both sparse and dense input features
* Support SIMD instructions (SSE, AVX256, and AVX512)
* Modularity implementation

## Comparison of PECOS and NMSLIB on the sparse data

#### Disclaimer 
The benchmarking results listed in this notebook are based on an `r5dn-24xlarge` AWS instance with 96 Intel(R) Xeon(R) Platinum 8259CL CPUs @ 2.50GHz. With distinct environments, the magnitude of improvments could be also different.

#### Results
* We compare two implementations of HNSW: `PECOS` and `NMSLIB` on a sparse dataset (i.e., RCV1).
* For RCV1, the instances in training/test set are `781,265` and `23,149`, respectively. The feature dimension is `47,236`.
* The HNSW index is constructed under `M=16` and `efConstruction=500`.
* From the table below, we see that, under similar Recall@10, `PECOS` achieves `[88%,93%]` speedup compared to the `NMSLIB` package.

| M=16, efC=500 |           |                         |    HNSW (PECOS)    |           |                         |    HNSW (NMSLIB)   | speedup (PECOS/NMSLIB) |
|:-------------:|:---------:|:-----------------------:|:------------------:|:---------:|:-----------------------:|:------------------:|:----------------------------:|
|      efS      | Recall@10 | Throughput (#query/sec) | Latency (ms/query) | Recall@10 | Throughput (#query/sec) | Latency (ms/query) |                              |
|            10 |    0.7733 |                5250.297 |             0.1905 |    0.7790 |                2710.256 |             0.3690 |                       93.72% |
|            20 |    0.8545 |                3677.292 |             0.2719 |    0.8581 |                1924.505 |             0.5196 |                       91.08% |
|            40 |    0.9043 |                2409.959 |             0.4149 |    0.9055 |                1271.085 |             0.7867 |                       89.60% |
|            80 |    0.9325 |                1508.349 |             0.6630 |    0.9326 |                 800.999 |             1.2484 |                       88.31% |
|           120 |    0.9434 |                1125.047 |             0.8889 |    0.9426 |                 597.873 |             1.6726 |                       88.17% |
|           200 |    0.9533 |                 763.752 |             1.3093 |    0.9523 |                 404.518 |             2.4721 |                       88.81% |
|           400 |    0.9621 |                 433.872 |             2.3048 |    0.9608 |                 229.553 |             4.3563 |                       89.01% |
|           600 |    0.9657 |                 305.747 |             3.2707 |    0.9644 |                 161.879 |             6.1775 |                       88.87% |
|           800 |    0.9678 |                 237.651 |             4.2078 |    0.9663 |                 124.806 |             8.0124 |                       90.42% |

## Hands-on Tutorial

The life cycle of a PECOS-HNSW model consists of two stages:

* building the indexer (training)
* inference (testing).

### Data Loading

In [1]:
! wget https://archive.org/download/pecos-dataset/ann-benchmarks/rcv1-angular-47236.tar.gz
! tar -zxvf ./rcv1-angular-47236.tar.gz

--2022-07-15 21:03:07--  https://archive.org/download/pecos-dataset/ann-benchmarks/rcv1-angular-47236.tar.gz
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia802308.us.archive.org/21/items/pecos-dataset/ann-benchmarks/rcv1-angular-47236.tar.gz [following]
--2022-07-15 21:03:07--  https://ia802308.us.archive.org/21/items/pecos-dataset/ann-benchmarks/rcv1-angular-47236.tar.gz
Resolving ia802308.us.archive.org (ia802308.us.archive.org)... 207.241.228.48
Connecting to ia802308.us.archive.org (ia802308.us.archive.org)|207.241.228.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 317972212 (303M) [application/octet-stream]
Saving to: ‘rcv1-angular-47236.tar.gz’

100%[======================================>] 317,972,212 11.0MB/s   in 40s    

2022-07-15 21:03:47 (7.68 MB/s) - ‘rcv1-angular-47236.tar.gz’ saved [317972212/3

In [2]:
import numpy as np
from pecos.utils import smat_util
X_trn = smat_util.load_matrix("./rcv1-angular-47236/X.trn.npz").astype(np.float32)
X_tst = smat_util.load_matrix("./rcv1-angular-47236/X.tst.npz").astype(np.float32)
Y_tst = smat_util.load_matrix("./rcv1-angular-47236/Y.tst.npy")
print("n_trn {:7d} n_tst {:7d} data_dim {:7d}".format(
    X_trn.shape[0], X_tst.shape[0], X_trn.shape[1])
)

n_trn  781265 n_tst   23149 data_dim   47236


### Training Indexer

To train a PECOS-HNSW model, training parameters need to be defined in an object of HNSW.TrainParams as the argument train_params. The key parameters of training a PECOS-HNSW model include:
* `M` (default 32): The maximum number of edges per node for each layer. A larger M leads to a larger model size and greater memory consumption. Higher/lower M are more suitable for high/low dimensional data or the pursue of high/low recall.
* `efC` (default 100): The size of the priority queue for best first search in construction. `efC` can be considered as the trade-off between efficiency and accuracy for indexing. A higher `efC` results in longer construction time but better quality of indexing.
* `metric_type` (default ip): The distance metric type for ANN search. PECOS-HNSW currently supports Euclidean distance (`l2`); and inner product (`ip`)
* `threads` (default -1): The number of threads for training, or -1 to use all available cores.

The parameters for inference can be also decided as the argument pred_params during model construction so that the model can be directly applied for inference without further parameter designation.


In [3]:
import time
from pecos.ann.hnsw import HNSW

M, efC = 32, 100
metric = "ip"
train_params = HNSW.TrainParams(
    M=M,
    efC=efC,
    metric_type=metric,
    threads=-1,
)
start_time = time.time()
model = HNSW.train(X_trn, train_params=train_params, pred_params=None)
print("HNSW Indexer | M {} efC {} metric {} | time(s) {}".format(
    M, efC, metric, time.time() - start_time),
)

HNSW Indexer | M 32 efC 100 metric ip | time(s) 11.980276823043823


### Save and Load Indexer

In [4]:
model_folder = "./rcv1.pecos-hnsw.index"
model.save(model_folder)
del model
model = HNSW.load(model_folder)

### Inference and Evaluation

To conduct inference with a train HNSW model, prediction parameters need to be defined in an object of HNSW.PredParams as the argument pred_params. The key parameters of inference with a PECOS-HNSW model include:

* `efS` (default 100): The size of the priority queue for best first search during inference. Similar to efC, efS can be considered as the trade-off between search efficiency and accuracy. A higher efS results in more accurate results with slower speed. efS is required to be greater than topk.
* `topk` (default 10): The number of approximate nearest neighbor to be returned. 
* `threads` (default -1): The number of searchers for parallel inference, -1 to use all available searchers.

The predict function derives the search results based on a query matrix of shape (# of data points for inference, # of dimentions) and `pred_params`, as well as searchers. The argument `ret_csr` (default `true`) decides the format of returned results as:

* If `ret_csr` is false, the returned results would be two matrices of shape (# of data points, topk), which indicate the topk indices in the training corpus and the corresponding distances for each testing instance.
* If `ret_csr` is true, the returned results would be a  [Compressed Sparse Row (CSR) matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) of shape (# of data points, # of points in the training corpus). Each row contains sorted topk distance values at the corresponding columns (i.e., indices in training corpus). The data for each row (i.e., `data[indptr[i]:indptr[i + 1]]`) are also sorted by the distance values.



In [5]:
pred_params = HNSW.PredParams(efS=100, topk=10)
searchers = model.searchers_create(num_searcher=1)
start_time = time.time()
indices, distances = model.predict(
    X_tst,
    pred_params=pred_params,
    searchers=searchers,
    ret_csr=False,
)
pred_time = time.time() - start_time
print(f"Prediction Time = {pred_time:.4f} seconds.")

Prediction Time = 15.7988 seconds.


### Evaluation

In [6]:
def compute_recall(neighbors, true_neighbors):
    total = 0
    for gt_row, row in zip(true_neighbors, neighbors):
        total += np.intersect1d(gt_row, row).shape[0]
    return total / true_neighbors.size

In [7]:
recall = compute_recall(indices, Y_tst)
throughput = indices.shape[0] / pred_time
latency = 1.0 / throughput * 1000.
print(f"HNSW inference | R@10 {recall:.4f} Throughput(q/s) {throughput:8.3f} latency(ms/q) {latency:8.4f}")

HNSW inference | R@10 0.9025 Throughput(q/s) 1465.236 latency(ms/q)   0.6825


## Recall vs Throughput Trade-off

In [8]:
def run_pecos(X_trn, X_tst, Y_tst):
    metric = "ip"
    M_list = [16]
    efC = 500
    topk = 10
    efS_list = [10, 20, 40, 80, 120, 200, 400, 600, 800]
    for M in M_list:
        train_params = HNSW.TrainParams(M=M, efC=efC, metric_type=metric, threads=-1)
        start_time = time.time()
        model = HNSW.train(X_trn, train_params=train_params, pred_params=None)
        print("Indexer | M {} efC {} metric {} | train time(s) {}".format(
            M, efC, metric, time.time() - start_time)
        )
        
        for efS in efS_list:
            pred_params = HNSW.PredParams(efS=efS, topk=topk)
            searchers = model.searchers_create(num_searcher=1)
            
            start_time = time.time()
            indices, distances = model.predict(X_tst, pred_params=pred_params, searchers=searchers, ret_csr=False)
            pred_time = time.time() - start_time
            
            recall = compute_recall(indices, Y_tst)
            throughput = indices.shape[0] / pred_time
            latency = 1.0 / throughput * 1000.
            print("inference | efS {:3d} R@10 {:.4f} Throughput(q/s) {:8.3f} latency(ms/q) {:8.4f}".format(
                efS, recall, throughput, latency)
            )

In [9]:
run_pecos(X_trn, X_tst, Y_tst)

Indexer | M 16 efC 500 metric ip | train time(s) 46.87919640541077
inference | efS  10 R@10 0.7733 Throughput(q/s) 5250.297 latency(ms/q)   0.1905
inference | efS  20 R@10 0.8545 Throughput(q/s) 3677.292 latency(ms/q)   0.2719
inference | efS  40 R@10 0.9043 Throughput(q/s) 2409.959 latency(ms/q)   0.4149
inference | efS  80 R@10 0.9325 Throughput(q/s) 1508.349 latency(ms/q)   0.6630
inference | efS 120 R@10 0.9434 Throughput(q/s) 1125.047 latency(ms/q)   0.8889
inference | efS 200 R@10 0.9533 Throughput(q/s)  763.752 latency(ms/q)   1.3093
inference | efS 400 R@10 0.9621 Throughput(q/s)  433.872 latency(ms/q)   2.3048
inference | efS 600 R@10 0.9657 Throughput(q/s)  305.747 latency(ms/q)   3.2707
inference | efS 800 R@10 0.9678 Throughput(q/s)  237.651 latency(ms/q)   4.2078


## Appendix: Install PECOS and NMSLIB

### Install via Conda 
```bash
conda create -n pecos-hnsw-tutorial python=3.8
conda activate pecos-hnsw-tutorial

pip install pyarrow pandas ipython jupyterlab
```

### Install PECOS from Source

We will install PECOS from source with the -march=native flag to optimize the best SIMD instruction available in your machine. More details available in https://github.com/amzn/pecos#installation-from-source

```bash
# prerequisite, assuming amazon linux 2 
sudo yum -y install python3 python3-devel python3-distutils python3-venv && sudo yum -y groupinstall 'Development Tools' 
sudo amazon-linux-extras install epel -y
sudo yum install openblas-devel -y
# pecos with -march=native flag
git clone https://github.com/amzn/pecos
cd pecos
PECOS_MANUAL_COMPILE_ARGS="-march=native" python -m pip install  --editable .
```

### Install NMSLIB from Source

We follow the install guide [install guide](https://github.com/erikbern/ann-benchmarks/blob/master/install/Dockerfile.nmslib) from ANN-Benchmark to install NMSLIB from source for the best performance.

```bash
# pre-requisite, assuming amazon linux 2
sudo yum -y install cmake boost-devel eigen3-devel
git clone https://github.com/searchivarius/nmslib.git
cd nmslib/similarity_search
cmake . -DWITH_EXTRAS=1
make -j4
pip install pybind11
cd ../python_bindings/
python setup.py build
python setup.py install
python -c 'import nmslib'
```

### Install via Docker (as in ANN-Benchmkark)

```bash
# install some basic stuff
sudo yum -y update
sudo yum install -y git curl zip unzip vim gcc-c++ htop

# https://docs.aws.amazon.com/AmazonECS/latest/developerguide/docker-basics.html
# sudo yum update -y
# sudo amazon-linux-extras install docker
sudo service docker start
sudo systemctl enable docker
sudo usermod -a -G docker ec2-user
docker info
```

### Install Docker Image

```bash
# install miniconda fist!
conda create -n ann-benchmarks python=3.8
conda activate ann-benchmarks

# install ANN package supported by ann-benchmarks
git clone https://github.com/erikbern/ann-benchmarks.git
cd ann-benchmarks
pip install -r requirements.txt

# install docker containers
python -u install.py --algorithm faiss
python -u install.py --algorithm hnswlib
python -u install.py --algorithm n2
python -u install.py --algorithm pecos
python -u install.py --algorithm scann
python -u install.py --algorithm ngt
python -u install.py --algorithm nmslib
python -u install.py --algorithm diskann
python -u install.py --algorithm pynndescent

# list all dockers
docker image ls
REPOSITORY TAG IMAGE ID CREATED SIZE
ann-benchmarks-hnswlib latest 2e1ea8d11df7 2 hours ago 1.04GB
ann-benchmarks-nmslib latest 1e094d3e96f7 3 hours ago 1.64GB
ann-benchmarks-faiss latest 44e5bd15bfcd 5 hours ago 4.9GB
ann-benchmarks-scann latest 5151abe3b09e 5 hours ago 2.76GB
ann-benchmarks latest c2c612131da4 5 hours ago 938MB
```

### Enter Docker Env

```bash
EFS_DIR=/PATH/TO/pecos-hnsw-kdd22
DOCKER_IMAGE=ann-benchmarks-nmslib

docker run --rm -it -v ${EFS_DIR}:/home/app/ws \
    --entrypoint /bin/bash ${DOCKER_IMAGE}
```